In [1]:
import pandas as pd
path = r'C:\Users\juan.sossa\projects_integrait\DNDI\data\db3bea42-add2-4185-9a08-d8a8a2622277.xlsx'
df_root = pd.read_excel(path)

In [2]:
df= df_root[df_root['name']=='Informed Consent']
lista_sujetos = df['Participante'].unique()
df = df[['name', 'Visit', 'activityState', 'Participante', 'Estado del Participante', 'Campo', 'Valor']]


In [3]:
df

,name,Visit,activityState,Participante,Estado del Participante,Campo,Valor
1039,Informed Consent,Screening Visit,DATA_EMPTY,011001,EARLY_TERMINATION,Informed consent signature date,-
1040,Informed Consent,Screening Visit,DATA_EMPTY,011004,ENROLLED,Informed consent signature date,NaN
1041,Informed Consent,Screening Visit,DATA_EMPTY,011005,ENROLLED,Informed consent signature date,NaN
1042,Informed Consent,Screening Visit,DATA_EMPTY,011006,EARLY_TERMINATION,Informed consent signature date,-
1043,Informed Consent,Screening Visit,DATA_EMPTY,S017,ENROLLED,Informed consent signature date,-
...,...,...,...,...,...,...,...
1102,Informed Consent,Screening Visit,DATA_ENTRY_COMPLETE,012013,ENROLLED,Protocol Version,1
1103,Informed Consent,Screening Visit,DATA_ENTRY_COMPLETE,012014,ENROLLED,Protocol Version,1
1104,Informed Consent,Screening Visit,DATA_ENTRY_COMPLETE,012015,ENROLLED,Protocol Version,1
1105,Informed Consent,Screening Visit,DATA_ENTRY_COMPLETE,012016,ENROLLED,Protocol Version,1


In [4]:
df_visit_date = df_root[df_root['name']=='Date of visit']
df_visit_date = df_visit_date[['Visit','Participante', 'Campo', 'Valor']]
df_visit_date = df_visit_date[df_visit_date['Campo']=='Visit Date']
df_visit_date = df_visit_date[['Visit','Participante','Valor']]
df_visit_date = df_visit_date.rename(columns={'Participante':'Subject'})

In [5]:
prueba = df[['Campo', 'Valor']].T
columns_analize = prueba.iloc[0].unique()
columns_analize

array(['Informed consent signature date',
       'Has the subject previously been screened in the current study?',
       'Prior screening number', 'Protocol Version'], dtype=object)

In [9]:
from datetime import datetime
from revision_fechas import revision_fecha
import warnings

warnings.filterwarnings('ignore')

lista_revision = []

# fecha_inicio = datetime.strptime('19-06-2023', "%d-%m-%Y")
# fecha_fin =  datetime.strptime('31-10-2023', "%d-%m-%Y")

lista_validacion_prior_screening = []
for sujeto in lista_sujetos:
    sujeto_principal = df[df['Participante']==sujeto]
    
    for visita in sujeto_principal.Visit.unique():
        pru_1 = sujeto_principal[sujeto_principal['Visit']==visita]
        pru = pru_1
        pru = pru[['Campo', 'Valor']].T
        new_columns = pru.iloc[0]
        pru = pru[1:].set_axis(new_columns, axis=1)
        pru['Subject'] = sujeto
        pru['Visit'] = visita
        pru['status'] = pru_1['activityState'].unique()
        pru = pru.merge(df_visit_date, on=['Subject', 'Visit'], how='left')
        

        for index, row in pru.iterrows():
            status = row['status']
            signature_date =  row['Informed consent signature date']

            try:
                previously_screened = row['Has the subject previously been screened in the current study?']
            except:
                pass
      
            prior_screening_number = row['Prior screening number']

            try:
                protocol_version  = row['Protocol Version']
            except:
                pass

            fecha_visita = row['Valor']
            subject = row['Subject']
            visit = row['Visit']

            date_format = '%d-%b-%Y'

            if status == 'INCOMPLETE' or status == 'PENDING_QUERIES':
                pass
            else:

                # try:
                #     date_visit = datetime.strptime(fecha_visita, date_format)
                #     date_ICF = datetime.strptime(signature_date, date_format)

                #     if date_ICF > date_visit:
                #         error = [subject, visit, 'The date of the ICF signature cannot be later than the date of the visit.' , (signature_date,fecha_visita ), 'IC0030']
                #         lista_revision.append(error)
                #     else:
                #         pass
                # except:
                #     pass

                f = revision_fecha(signature_date)
                if f == None:
                    pass
                else:
                    error = [subject, visit, f , signature_date, 'GE0020']
                    lista_revision.append(error)

                if prior_screening_number[0] != 'S' or len(prior_screening_number) == 4:
                    error = [subject, visit, 'The screening number format has to be (S000)' , prior_screening_number, 'IC0020']
                    lista_revision.append(error)
                else:
                    pass
                
                if  prior_screening_number in lista_validacion_prior_screening:
                    error = [subject, visit, 'The entered number should be a non existing subject number' , prior_screening_number, 'IC0020']
                    lista_revision.append(error)
                else:
                    pass

                lista_validacion_prior_screening.append(prior_screening_number)
            

In [7]:
lista_revision

[['011001', 'Screening Visit', 'Value is not a valid date', '-', 'GE0020'],
 ['011001',
  'Screening Visit',
  'The screening number format has to be (S000)',
  '-',
  'IC0020'],
 ['011004', 'Screening Visit', 'Value is not a valid date', nan, 'GE0020'],
 ['011004',
  'Screening Visit',
  'The screening number format has to be (S000)',
  '-',
  'IC0020'],
 ['011004',
  'Screening Visit',
  'The entered number should be a non existing subject number',
  '-',
  'IC0020'],
 ['011005', 'Screening Visit', 'Value is not a valid date', nan, 'GE0020'],
 ['011005',
  'Screening Visit',
  'The screening number format has to be (S000)',
  '-',
  'IC0020'],
 ['011005',
  'Screening Visit',
  'The entered number should be a non existing subject number',
  '-',
  'IC0020'],
 ['011006', 'Screening Visit', 'Value is not a valid date', '-', 'GE0020'],
 ['011006',
  'Screening Visit',
  'The screening number format has to be (S000)',
  '-',
  'IC0020'],
 ['011006',
  'Screening Visit',
  'The entered nu